In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
import os
from keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import numpy as np
from skimage import data, color, feature
from itertools import chain

In [34]:
categories = ['glioma', 'meningioma', 'notumor', 'pituitary']

In [35]:
def pre_process():
  base_path = '/content/drive/MyDrive/Machine Learning Assignment/Dataset/'
  types = ['Training', 'Testing']
  x_train = []
  x_test = []
  y_train = []
  y_test = []

  for type_ in types:
    for i, catagory in enumerate(categories):
      partial_path = os.path.join(base_path, type_, catagory)
      images = os.listdir(partial_path)
      for img in images:
        full_path = os.path.join(partial_path, img)
        if type_ == types[0]:
          x_train.append(img_to_array(load_img(full_path, color_mode='grayscale', target_size=(64, 64))))
          y_train.append(i)
        elif type_ == types[1]:
          x_test.append(img_to_array(load_img(full_path, color_mode='grayscale', target_size=(64, 64))))
          y_test.append(i)

  x_train = np.array([feature.hog(img) for img in chain(x_train)])
  x_test = np.array([feature.hog(img) for img in chain(x_test)])

  y_train = np.array(y_train)
  y_test = np.array(y_test)

  return x_train, x_test, y_train, y_test

In [36]:
x_train, x_test, y_train, y_test = pre_process()

In [47]:
x_train.shape

(5732, 2916)

In [38]:
model = SVC()
model.fit(x_train, y_train)

SVC()

In [39]:
model.score(x_test, y_test)

0.8977879481311976

In [46]:
img = load_img('/content/drive/MyDrive/Machine Learning Assignment/Dataset/Testing/glioma/Te-gl_0028.jpg', color_mode='grayscale', target_size=(64, 64))
img = feature.hog(img).reshape((1, -1))
categories[model.predict(img)[0]]

'glioma'

In [48]:
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/SVC90_model.pickle', 'wb') as handle:
  pickle.dump(model, handle)